In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from random import sample

### Định nghĩa gene

In [ ]:
class Gene: # gene
#     __distances_table = {}
  def __init__(self, location):
    self.name = location[0]
    self.x_value = location[1]
    self.y_value = location[2]
    self.z_value = location[3]
        
  def get_distance_to(self, dest):
    x_origin, y_origin, z_origin = self.x_value, self.y_value, self.z_value
    x_dest, y_dest, z_dest = dest.x_value, dest.y_value, dest.z_value
    return ((x_dest-x_origin)**2 + (y_dest-y_origin)**2 + (z_dest-z_origin)**2)**0.5
    
  def get_gene(self): 
    return (self.x_value, self.y_value, self.z_value)

  def get_name(self):
    return self.name

# cá thể
class Individual:
  def __init__(self, genes):
    self.genes = genes.copy()
    self.__cost = 0

    self.name = [gene.get_name() for gene in genes]
    
  @property
  def cost(self):
      self.__cost = Energy_consumption_sensor(self)
      return self.__cost
  def get_individual(self):
    #######
    return [gene.get_gene() for gene in self.genes]
  def copy(self):
    return Individual(self.genes.copy())

  # Quần thể
class Population:
  def __init__(self, individuals):
    self.individuals = individuals
    self.__length = len(individuals)
    self.__get_fittest = 0
        
  @property
  def length(self):
    self.__length = len(self.individuals)
    return self.__length
    
  def add(self, indivs):
    individuals = self.individuals.copy()
    for indiv in indivs:
      individuals.append(indiv.copy())
    self.individuals = individuals
    
  def random_indivs(self, nums_indiv):
    genes = self.individuals[0].genes.copy()
    individuals = []
    for _ in range(nums_indiv):
      np.random.shuffle(genes)
      individuals.append(Individual(genes.copy()))
    self.add(individuals)

  def remove(self, indivs):
    for indiv in indivs:
      self.individuals.remove(indiv)
    
  def get_fittest(self):
    if(self.__get_fittest == 0):
      best_indiv = self.individuals[0].copy()
      individuals = self.individuals
      for indiv in individuals:
        a = indiv.copy().cost
        b = best_indiv.copy().cost
        if(b > a):
          best_indiv = indiv.copy()
      self.__get_fittest = best_indiv
    return self.__get_fittest
  
  # Chọn lọc các thể
  def selection(self, max_len_indiv):
    if(len(self.individuals) <= max_len_indiv):
      pass
    else:
      individuals = self.individuals
      arr_cost = [(i, individuals[i].cost) for i in range(len(individuals))]
      arr_cost.sort(key=lambda x: x[1])
      index_cost = [i[0] for i in arr_cost[:max_len_indiv]]
      self.individuals = [individuals[i] for i in index_cost]
    self.__get_fittest = self.individuals[0].copy()

### Lai ghép và đột biến

In [ ]:
def crossover_mutate(parent1, parent2, pc=0.7, pm=0.2):
  result = []

  # lai ghép
  rc = np.random.random()
  if(rc < pc):
    t = sample(range(len(parent1.genes)), 2)
    t.sort()
    [t1, t2] = t
    child1 = parent1.genes.copy()
    child2 = parent2.genes.copy()

    child1[t1:t2] = parent2.genes[t1:t2]
    child2[t1:t2] = parent1.genes[t1:t2]
    len_gene = len(parent1.genes)

    for i in range(t1):
      while(child1[i] in child1[t1:t2]):
        child1[i] = parent1.genes[parent2.genes.index(child1[i])]

      while(child2[i] in child2[t1:t2]):
        child2[i] = parent2.genes[parent1.genes.index(child2[i])]
    
    for i in range(t2, len_gene):
      while(child1[i] in child1[t1:t2]):
        child1[i] = parent1.genes[parent2.genes.index(child1[i])]

      while(child2[i] in child2[t1:t2]):  
        child2[i] = parent2.genes[parent1.genes.index(child2[i])]

    result.append(Individual(child1))
    result.append(Individual(child2))
  
  # Đột biến
  rm = np.random.random()
  if(rm < pm):
    child1 = mutate(parent1)
    child2 = mutate(parent2)
    result.append(child1)
    result.append(child2)
  return result

def mutate(indiv):
  genes = indiv.genes.copy()
  a, b = sample(list(range(len(genes))), 2)
  
  # swap
  temp = genes[a]
  genes[a] = genes[b]
  genes[b] = temp
  return Individual(genes)

def evolution(pop, pc=0.7, pm=0.2, MAX_LEN=100):
  indivs_child = []
  while(len(indivs_child) < MAX_LEN):
    try:
      parent1, parent2 = sample(pop.individuals, 2)
      indivs_child += crossover_mutate(parent1, parent2, pc, pm)
    except:
      print("individuals in population < 2")
  
  pop.add(indivs_child)
  pop.selection(MAX_LEN)

### Dữ liệu

In [ ]:
f = open("/content/drive/MyDrive/data/ga-dem10_r25_1_0.json")
data = json.load(f)
f.close()

num_of_relays = data['num_of_relays']
num_of_sensors = data['num_of_sensors']

relays = data['relays']
sensors = data['sensors']
center = data['center']
radius = data['radius']

list_node = [center] + relays + sensors
nodes = []
for i in range(len(list_node)):
  nodes.append([i, list_node[i]['x'], list_node[i]['y'], list_node[i]['z']])

### Các hàm decoding

In [ ]:
def matrix_edge_adjacents(G):
  """
    return {
      'node': [node1, node2, ...],,
      ....
    }
  """
  V1, E1 = G
  result = np.ones((len(v1), len(v1)))
  for node in V1:
    adjacents = []
    for edge in E1:
      if(edge[0] == node):
        adjacents.append(edge[1])
      if(edge[1] == node):
        adjacents.append(edge[0])
    result[node] = adjacents
  return result

def get_adjacent(s, E_):
  arr = []
  for i in range(len(E_[0])):
    if(E_[i][s] + E_[s][i] >= 1):
      arr.append(i)
  return arr

def check_cycle(edge, G_):
  (V_ , E_) = G_
  (u, s) = edge
  visit = {}
  for node in V_:
    visit[node] = 0

  queue = get_adjacent(node, E_)
  
  if(u in queue):
    return True

  while(len(queue) > 0):
    node = queue.pop(0)
    if(visit[node] == 1):
      continue
    visit[node] = 1
    adj_temp = get_adjacent(node, E_)
    queue += adj_temp
    if(u in queue):
      return True
  return False

def decoding_network(G, relays, sensors, r, indiv_name):
  """
    G(V, E): V: array relay, sensor node, E array edges [(a,b),..]
    relays: arrry relays node
    sensors: arrry sensor node
    r: number relay selected (r <= n_relay)
    indiv_name: hoán vị của sensors, relays
  """
  (V, E) = G
  m, n = len(relays), len(sensors)

  V_t = [0] # 0 is node source
  E_t = [[0 for i in range(len(genes))] for j in range(len(genes))]
  E_result = []
  relays.sort(key=lambda x: indiv_name.index(x))
  for rn in relays[:r]:
    V_t.append(rn)
    E_t[0][rn] = 1
    E_result.append((0, rn))

  sensors.sort(key=lambda x: indiv_name.index(x))
  V_t += sensors
  for s in sensors:
      adj = get_adjacent(s, E_metrix)
      adj.sort(key=lambda x: indiv_name.index(x))
      for u in adj:
        if (u in V_t) and (check_cycle((u, s), (V_t, E_t)) == False):
          E_t[u][s] = 1
          E_result.append((u,s))
          break

  if(len(E_t) < len(sensors)+r):
    return ([0], [])
  return (V_t, E_result)

In [ ]:
e_elec = 50*(10**-9)
e_fs = 10*(10**-12)
e_mp = 0.0013*(10**-12)
e_da = 5*(10**-12)
d0 = (e_fs/e_mp)**0.5
MAX_ENERRY = 10**5
do_sau = 5
k_bit = 4000

def E_distance(node, nodes_parent, nodes_child):
  if(node not in nodes_child):
    return MAX_ENERRY
  else:
    # edge
    node_parent = nodes_parent[nodes_child.index(node)]
    d = genes[node].get_distance_to(genes[node_parent])

    # check độ sâu
    dem = 2
    while(node_parent != 0):
      node_parent = nodes_parent[nodes_child.index(node_parent)]
      dem += 1
      if(dem > do_sau):
        return MAX_ENERRY
    
    if(dem > do_sau):
      return MAX_ENERRY

    if(d <= d0):
      return (e_elec + e_fs*d*d)
    elif(d <= radius):
      return (e_elec + e_mp*(d**4))
    else:
      return MAX_ENERRY

def Energy_consumption_sensor(indiv):
  indiv_name = indiv.name.copy()
  E_max = 0
  (V_t, E_t) = decoding_network(G, relays_node, sensors_node, r, indiv_name)
  nodes_parent = [edge[0] for edge in E_t]
  nodes_child = [edge[1] for edge in E_t]
  for node in V_t[1:]:
    E_node = 0
    if(node in sensors_node):
      num = nodes_parent.count(node)
      # print("{0}: {1}".format(node, num))
      E_node = num*e_elec + (num+1)*e_da + E_distance(node, nodes_parent, nodes_child)
    elif(node in relays_node):
      num = nodes_parent.count(node)
      # print("{0}: {1}".format(node, num))
      E_node = num*(e_elec + e_da) + E_distance(node, nodes_parent, nodes_child)
    
    E_max = max(E_node, E_max)
  return E_max

### Tạo đồ thị G(V, E) và các hằng số liên quan

In [ ]:
genes = [Gene(node) for node in nodes]
V = list(range(len(genes)))
E = []
E_metrix = [[0 for i in range(len(genes))] for j in range(len(genes))]
for i in range(len(genes)-1):
  for j in range(i+1,len(genes)):
    if(genes[i].get_distance_to(genes[j]) <= radius):
      E_metrix[i][j] = 1
      E_metrix[j][i] = 1
      E.append((i, j))

relays_node = V[1:num_of_relays+1]
sensors_node = V[num_of_relays+1:]
G = (V, E)
r = 40
# ADJACENTS = matrix_edge_adjacents(G)

In [ ]:
pop = Population([Individual(genes[1:])])
pop.random_indivs(199)
pop.remove([pop.individuals[0]])

In [ ]:
# arr = []
for i in range(300):
  evolution(pop, 0.3, 1, 200)
  arr.append(pop.get_fittest().cost*4000)
  print("Lan thu {0}: {1}".format(i, arr[-1]))

Lan thu 0: 0.0016093138480070416
Lan thu 1: 0.0016093138480070416
Lan thu 2: 0.0016093138480070416
Lan thu 3: 0.0014163853350217083
Lan thu 4: 0.0014163853350217083
Lan thu 5: 0.0014163853350217083
Lan thu 6: 0.0014163853350217083
Lan thu 7: 0.0014163853350217083
Lan thu 8: 0.0014163853350217083
Lan thu 9: 0.0014066701533983646
Lan thu 10: 0.0014066701533983646
Lan thu 11: 0.0014066701533983646
Lan thu 12: 0.0014066701533983646
Lan thu 13: 0.0014066701533983646
Lan thu 14: 0.0014066701533983646
Lan thu 15: 0.0014066701533983646
Lan thu 16: 0.0014066701533983646
Lan thu 17: 0.0012220859043596017
Lan thu 18: 0.0012214981723655423
Lan thu 19: 0.0012204936385975275
Lan thu 20: 0.001219850689032926
Lan thu 21: 0.0012167006472308358
Lan thu 22: 0.0012167006472308358
Lan thu 23: 0.0012167006472308358
Lan thu 24: 0.0010214781723655426
Lan thu 25: 0.001018185659556847
Lan thu 26: 0.001018185659556847
Lan thu 27: 0.0010174425122923146
Lan thu 28: 0.0010174425122923146
Lan thu 29: 0.0010174425122

In [ ]:
arr

[0.0026774844211824747,
 0.0026774844211824747,
 0.0020567968670248456,
 0.0020567968670248456,
 0.0020567968670248456,
 0.0020567968670248456,
 0.0020567968670248456,
 0.0020567968670248456,
 0.0020567968670248456,
 0.0020567968670248456,
 0.0020567968670248456,
 0.0020567968670248456,
 0.0018410519885365113,
 0.0018410519885365113,
 0.0018410519885365113,
 0.0018410519885365113,
 0.0018410519885365113,
 0.0018410519885365113,
 0.0018410519885365113,
 0.0018410519885365113,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.001828881058926521,
 0.0016817087576137956,
 0.0016817087576137956,
 0.0016817087576137956,
 0.0016817087576137956,
 0.0016817087576137956,
 0.0016817087576137956,
 0.0016817087576137956,
 0.0016817087576137956,
 0.001